# Process Kaiser Health Data

This script adds the Kaiser Health data with population, population aged 60+, and ICU beds for each county. It skips counties for which this data is missing and merges by FIPS code using the county to FIPS lookup table generated by `000_process_nytimes`.

The Kaiser population columns are labelled, in order to incorporate US Census data as well.

Kaiser Data sourced from [here](https://khn.org/news/as-coronavirus-spreads-widely-millions-of-older-americans-live-in-counties-with-no-icu-beds/#lookup).

In [1]:
# change this variable to the table of processed NYTimes data
NYTIMES_DATA = "nytimes_processed_70days.csv"

import pandas as pd

# import all needed data
nytimes_df = pd.read_csv(NYTIMES_DATA, index_col=0)
fips_county_df = pd.read_csv("fips_county_lookup.csv")
kaiser_df = pd.read_csv("kaiser_health_data.csv")

This next cell does the bulk of the processing:

In [2]:
# store updated rows in this list
new_df_rows = []

# this loops through the existing data from the NYTimes processing,
# and adds the kaiser data
for index, row in nytimes_df.iterrows():
    try:
        # find the county for this row of the data, and the corresponding row of Kaiser data
        county_lookup = fips_county_df.loc[fips_county_df.fips == int(row.fips)].iloc[0]
        kaiser_row = kaiser_df.loc[(kaiser_df["County"] == county_lookup.county) & (kaiser_df["State"] == county_lookup.state)].iloc[0]

        # add the new data to the row
        row["icu_beds"] = kaiser_row["ICU Beds"]
        row["kaiser_total_population"] = kaiser_row["Total Population"]
        row["kaiser_60plus_population"] = kaiser_row["Population Aged 60+"]

        # save this row to the list
        row.fips = int(row.fips)
        new_df_rows.append(row)
    except IndexError as e:
        # this county isn't in the Kaiser data, for now just skip
        pass

Finally, save the list of rows to a `DataFrame` and export to csv:

In [3]:
kaiser_processed_df = pd.DataFrame(new_df_rows)
kaiser_processed_df.to_csv(NYTIMES_DATA.replace("nytimes", "kaiser"))